## Setting up 

To run the code in colab, mount the drive and install cvxpy package.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%pip install cvxpy

In [ ]:
import numpy as np
from sklearn.random_projection import GaussianRandomProjection
import scipy.spatial
import math
import sys
import cvxpy as cp
from numpy import linalg as LA

## Setting up the random projection method
We use the projection matrix with the i.i.d. Gaussian entries. The input is the metric space, given as the matrix of hig-dimesnioal vectors in the rows, and the traget dimension $k$. The ouput is the $k$ - dimensional representation of the input space, stored as the rows in the output matrix.


In [ ]:
def JL_transf(space, k):
    transformer = GaussianRandomProjection(k)
    result=transformer.fit_transform(space)
    return(result)


We also implement a small function that adds a row of zeros to an input matrix. We will use it in our approximation algorithm. 

In [ ]:
def add_first_zero_row(matrix):
    [size, dim]=matrix.shape
    new_matrix=np.zeros((size+1, dim))
    for i in range(1,size+1):
        for j in range(dim):
            new_matrix[i,j]=matrix[i-1,j]
    return(new_matrix);

## Approximation algorithm


**Input:** a metric space on $n$ points, given as a matrix of pairwise Euclidean distances; a traget dimension $k \geq 3$; parameter $q$ desribing the moment of the distorin to be approximated. 

**Output:** the set of $k$-dimensional vectors, such that their pairwise Euclidean distances are preserved up to a small multiplicative error, on average. I.e., an embedding of the input distances into $k$-dimensions with near optimal $l_q$-distortion.


In [ ]:
def Approx_Algo(input_dists, new_dim, q):
    [rows, cols]=input_dists.shape
    #Step1: convex optimization, using the implementation in package cvxpy.

    #Normalize the input metric by the largest dsiatnce (this should not change the optimal embedding,
    #but this helps to speed up the computations and make them more precise).
    max_dist=np.amax(input_dists)
    div_input_dists=np.divide(input_dists, max_dist)

    G=cp.Variable((rows-1,rows-1), PSD=True)

    #Z i sthe matrix of the new dists, squared
    Z=cp.Variable((rows,rows),symmetric=True)

    #E is the matrix of expansions, squared
    E=cp.Variable((rows, rows), symmetric=True)

    #C is the matrix of contractions, squared
    C=cp.Variable((rows, cols), symmetric=True)
    C=cp.inv_pos(E)

    #M is the matrix of distortions, squared
    M=cp.Variable((rows, cols),symmetric=True)
    M=cp.maximum(E, C)


    one=cp.Parameter()
    one.value=1

    #the constraints describe the convex boundary set
    constraints=[]
    for j in range(1,rows):
        constraints=constraints+[Z[0,j]==G[j-1,j-1]]

    for i in range(1, rows):
        for j in range(i+1,rows):
            constraints=constraints+[Z[i,j]==G[i-1,i-1]+G[j-1,j-1]-2*G[i-1,j-1]]

    for i in range(rows):
        for j in range(i+1,rows):
            constraints=constraints+[Z[i,j]>=0]

    for i in range(rows):
            constraints=constraints+[E[i,i]==one]


    for i in range (rows):
        for j in range (i+1, rows):
            constraints=constraints+[Z[i,j]==E[i,j]*(div_input_dists[i,j]**2)]


    #The optimization objective function is l_q-distortion.
    prob=cp.Problem(cp.Minimize(cp.Pnorm(M, p=q/2)),constraints)
    prob.solve(verbose=True)
    
    #After the optimiation step, the matrix G contains the optimal pairwise Euclidean distances
    #approximating original pairwise distances. 

    #Recovering the resulting vectors of the embedding from the distances, by computing eigenvalue decomposition of G.
    eig_vals, eig_vectors=np.linalg.eigh(G.value)
    num_eigs=len(eig_vals)
    D_matrix=np.zeros((num_eigs, num_eigs))
    for i in range(num_eigs):
         D_matrix[i,i]=math.sqrt(abs(eig_vals[i]))

    #The rows of U should be the orthonormal basis of the eig_vectors.
    U_matrix=np.transpose(eig_vectors)
    the_vectors=np.matmul(D_matrix, U_matrix)

    #The original vectors are the cols of the above matrix.
    recov_vecs=np.transpose(the_vectors)

    #The assumption is that the first vector is mapped to 0 vector. So we bring it back.
    vectors=add_first_zero_row(recov_vecs)


    #Note:  We could use the Cholesky decomposition of python,
    #but there are floating point issues, so we implemented our own decomposition.

    #Step 2: embed the high dimimensional vectors into vectors of dimension new_dim, with the JL projection.
    #Output is the set of k-dimensional vectors.
 

    low_dim_space=JL_transf(vectors, new_dim)

    #Bring the normalization factor back.
    real_low_dim_space=low_dim_space*max_dist
    return(real_low_dim_space);


## Generating random non-Euclidean metric spaces and Evaluation functions 

In the papaer we test our implementation on the synthetically generated data set $X$ of the following form. We first randomly sample $n=100$ vectors, each of dimension $d=100$. This forms a Euclidean metric space. We then add a small random noise to the pairwisw distances in such a way that the reuslting distances represent a valid **non-Euclidean** metric space. 
 


In [ ]:
def get_random_space(size, dim):
    space=np.zeros((size, dim))
    for i in range(size):
        sdv=np.random.randint(1,30)
        for j in range(dim):
            space[i,j]=np.random.normal(0,sdv)
    return(space);

We also need a function that computes the pairwise distances of a given metic space: 

In [ ]:
def space_to_dist(space):
    dist=scipy.spatial.distance.pdist(space,metric='euclidean')
    matrix_dist=scipy.spatial.distance.squareform(dist)
    #answer=np.around(matrix_dist,8)
    #print("The distances are", matrix_dist)
    return (matrix_dist);

In [ ]:

#Generates a metric space that is "epsilon-far" from a given space. Input: distances matrix of a given m. space. Output: distances matrix.
#The resulted metric space itself is not Euclidean. Returns the distances matrix. The algorithm is randomized and it always outputs the metric space.
#There is some small probability that the output space is still Euclidean space (if the input space was Euclidean). To reduce this probability we run
#the algorithm for several iterations (#iter.)
#NOTE: for some runs, the result of is_metric_space(output) can result in False, due to rounding issues.

def get_epsilon_far_metric(dists_matrix, epsilon, iter):
    copy_dists_matrix=np.copy(dists_matrix)
    [rows, cols]=dists_matrix.shape

    #COMMENTS: the distorted metric space
    generated_metric_dists=np.zeros((rows, cols))
    for i in range(rows):
        for j in range(i+1, rows):
            lower_range=[]
            upper_range=[]
            for k in range(rows):
                if (k!=i and k!=j):
                    min_z=min(copy_dists_matrix[i,k], copy_dists_matrix[j,k])
                    max_z=max(copy_dists_matrix[i,k], copy_dists_matrix[j,k])
                    lower_range.append(max_z-min_z)
                    upper_range.append(max_z+min_z)
                    continue
            lower_array=np.array(lower_range)
            upper_array=np.array(upper_range)
            min_new=np.amax(lower_range)
            max_new=np.amin(upper_range)
            r=copy_dists_matrix[i,j]
            Finish=False
            possible_new_dists=[]
            for t in range(iter):
                noise=np.random.normal(0, epsilon)
                if (noise>=0):
                    factor=1+noise
                else:
                    factor=1/(1-noise)
                r_new=factor*r
                if (r_new>=min_new and r_new<=max_new):
                    Finish=True
                    possible_new_dists.append(r_new)
            if(Finish==True):
                new_dist=possible_new_dists[0]
            else:
                new_dist=min_new
            generated_metric_dists[i,j]=new_dist
            generated_metric_dists[j,i]=new_dist
            copy_dists_matrix[i,j]=new_dist
            copy_dists_matrix[j,i]=new_dist
    return(generated_metric_dists)




In the next code we just run the above piece as a subrutine, until we get the desured non-Euclidean metric space. We need a function that checks whether a given metric space is a Euclidean space:

In [ ]:
#input: squared dists of the metric space
def is_Euclidean_space(sq_dists):
    #assuming the dists_space is of a metric space of size=n
    [rows,cols]=sq_dists.shape
    G_matrix=np.zeros((rows, cols))
    for i in range(1,rows):
        for j in range(1,cols):
            G_matrix[i,j]=(1/2)*(sq_dists[0,i]+sq_dists[0,j]-sq_dists[i,j])
    G_row_del=np.delete(G_matrix,0,0)
    final_matrix=np.delete(G_row_del,0,1)
    answer=is_pos_def(final_matrix)
    return(answer);

def is_pos_def(X):
    return (np.all(np.linalg.eigvalsh(X) >= 0));    

In [ ]:
#COMMENTS: from our random space, loop the above code until you get a non-Euclidean result
def get_random_epsilon_far_non_Eucl(n, epsilon):
    original=get_random_space(n,n)
    original_Eucl_dists=space_to_dist(original)
    distorted_dists=get_epsilon_far_metric(original_Eucl_dists, epsilon, 5)
    while(is_Euclidean_space(distorted_dists**2)==True):
        original=get_random_space(n,n)
        original_Eucl_dists=space_to_dist(original)
        distorted_dists=get_epsilon_far_metric(original_Eucl_dists, epsilon, 2)
    return(distorted_dists)

We are now ready to generate the smaple non-Euclidean metric space $X$, of size $n=100$


In [ ]:
X=get_random_epsilon_far_non_Eucl(100, 0.8)
#test for being non-Euclidean
print("Is X a Euclidean metric?", is_Euclidean_space(X**2)==True)

Is X a Euclidean metric? False


The following code computes $l_q$-distortion of an embedding of an input metric into an output metric: 

In [ ]:
#l_q distortion measure
def lq_dist(input_dist, embedded_dist, q):
    [rows, cols]=input_dist.shape
    answer=0
    pairs=scipy.special.binom(rows, 2)
    for i in range (rows):
        for j in range(i+1,cols):
            curr=distortion(input_dist[i,j], embedded_dist[i,j])       
    return(((answer/pairs))**(1/float(q)));


def expans(old, new):
    return(new/old);    

def contr(old, new):
    if(new==0):
       sys.exit("Contraction of the pair is infinite!");
    return(old/new);    

def distortion(old, new):
    expansion=expans(old, new)
    contraction=contr(old,new)
    if (expansion>= contraction):
        distort=expansion
    else:
        distort=contraction
    return(distort);


Now you are set up to play with the data and the algorithm! Enjoy!

In [ ]:
def test_Approx_Algo():
  "Your code for testing"
  return 